# Description

This notebook compiles information about the GWAS and TWAS for a particular cohort. For example, the set of GWAS variants, variance of predicted expression of genes, etc.

It has specicfic parameters for papermill (see under `Settings` below).

This notebook should not be directly run. It is used by other notebooks.

# Modules

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
import pickle

import numpy as np
import pandas as pd

import conf
from entity import Gene

# Settings

In [3]:
# a cohort name (it could be something like UK_BIOBANK, etc)
COHORT_NAME = None

# reference panel such as 1000G or GTEX_V8
REFERENCE_PANEL = "GTEX_V8"

# predictions models such as MASHR or ELASTIC_NET
EQTL_MODEL = "MASHR"

# a string with a path pointing to an imputed GWAS
GWAS_FILE = None

# a string with a path pointing where S-PrediXcan results (tissue-specific are located
SPREDIXCAN_FOLDER = None

# an f-string with one placeholder {tissue}
SPREDIXCAN_FILE_PATTERN = None

# a string with a path pointing to an S-MultiXcan result
SMULTIXCAN_FILE = None

# output dir
OUTPUT_DIR_BASE = None

In [4]:
# Parameters
PHENOPLIER_NOTEBOOK_FILEPATH = "projects/asthma-copd/nbs/20_gene_corrs/jobs/01-compile_gwas_snps_and_twas_genes.ipynb"
COHORT_NAME = "asthma_only"
GWAS_FILE = "/opt/data/projects/asthma-copd/results/final_imputed_gwas/GWAS_Asthma_only_GLM_SNPs_info0.7-harmonized-imputed.txt.gz"
SPREDIXCAN_FOLDER = "/opt/data/projects/asthma-copd/results/twas/spredixcan"
SPREDIXCAN_FILE_PATTERN = "GWAS_Asthma_only_GLM_SNPs_info0.7-gtex_v8-mashr-{tissue}.csv"
SMULTIXCAN_FILE = "/opt/data/projects/asthma-copd/results/twas/smultixcan/GWAS_Asthma_only_GLM_SNPs_info0.7-gtex_v8-mashr-smultixcan.txt"
OUTPUT_DIR_BASE = "/opt/data/projects/asthma-copd/results/gls_phenoplier"


In [5]:
assert COHORT_NAME is not None and len(COHORT_NAME) > 0, "A cohort name must be given"

COHORT_NAME = COHORT_NAME.lower()
display(f"Cohort name: {COHORT_NAME}")

'Cohort name: asthma_only'

In [6]:
assert (
    REFERENCE_PANEL is not None and len(REFERENCE_PANEL) > 0
), "A reference panel must be given"

display(f"Reference panel: {REFERENCE_PANEL}")

'Reference panel: GTEX_V8'

In [7]:
assert GWAS_FILE is not None and len(GWAS_FILE) > 0, "A GWAS file path must be given"
GWAS_FILE = Path(GWAS_FILE).resolve()
assert GWAS_FILE.exists(), "GWAS file does not exist"

display(f"GWAS file path: {str(GWAS_FILE)}")

'GWAS file path: /opt/data/projects/asthma-copd/results/final_imputed_gwas/GWAS_Asthma_only_GLM_SNPs_info0.7-harmonized-imputed.txt.gz'

In [8]:
assert (
    SPREDIXCAN_FOLDER is not None and len(SPREDIXCAN_FOLDER) > 0
), "An S-PrediXcan folder path must be given"
SPREDIXCAN_FOLDER = Path(SPREDIXCAN_FOLDER).resolve()
assert (
    SPREDIXCAN_FOLDER.exists()
), f"S-PrediXcan folder does not exist: {str(SPREDIXCAN_FOLDER)}"

display(f"S-PrediXcan folder path: {str(SPREDIXCAN_FOLDER)}")

'S-PrediXcan folder path: /opt/data/projects/asthma-copd/results/twas/spredixcan'

In [9]:
assert (
    SPREDIXCAN_FILE_PATTERN is not None and len(SPREDIXCAN_FILE_PATTERN) > 0
), "An S-PrediXcan file pattern must be given"
assert (
    "{tissue}" in SPREDIXCAN_FILE_PATTERN
), "S-PrediXcan file pattern must have a '{tissue}' placeholder"

display(f"S-PrediXcan file template: {SPREDIXCAN_FILE_PATTERN}")

'S-PrediXcan file template: GWAS_Asthma_only_GLM_SNPs_info0.7-gtex_v8-mashr-{tissue}.csv'

In [10]:
assert (
    SMULTIXCAN_FILE is not None and len(SMULTIXCAN_FILE) > 0
), "An S-MultiXcan result file path must be given"
SMULTIXCAN_FILE = Path(SMULTIXCAN_FILE).resolve()
assert (
    SMULTIXCAN_FILE.exists()
), f"S-MultiXcan result file does not exist: {str(SMULTIXCAN_FILE)}"

display(f"S-MultiXcan file path: {str(SMULTIXCAN_FILE)}")

'S-MultiXcan file path: /opt/data/projects/asthma-copd/results/twas/smultixcan/GWAS_Asthma_only_GLM_SNPs_info0.7-gtex_v8-mashr-smultixcan.txt'

In [11]:
assert (
    EQTL_MODEL is not None and len(EQTL_MODEL) > 0
), "A prediction/eQTL model must be given"

display(f"eQTL model: {EQTL_MODEL}")

'eQTL model: MASHR'

In [12]:
assert (
    OUTPUT_DIR_BASE is not None and len(OUTPUT_DIR_BASE) > 0
), "Output directory path must be given"

OUTPUT_DIR_BASE = (Path(OUTPUT_DIR_BASE) / "gene_corrs" / COHORT_NAME).resolve()

OUTPUT_DIR_BASE.mkdir(parents=True, exist_ok=True)

display(f"Using output dir base: {OUTPUT_DIR_BASE}")

'Using output dir base: /opt/data/projects/asthma-copd/results/gls_phenoplier/gene_corrs/asthma_only'

# Load MultiPLIER Z genes

In [13]:
multiplier_z_genes = pd.read_pickle(
    conf.MULTIPLIER["MODEL_Z_MATRIX_FILE"]
).index.tolist()

In [14]:
len(multiplier_z_genes)

6750

In [15]:
assert len(multiplier_z_genes) == len(set(multiplier_z_genes))

In [16]:
multiplier_z_genes[:5]

['GAS6', 'MMP14', 'DSP', 'MARCKSL1', 'SPARC']

# GWAS

In [17]:
gwas_file_columns = pd.read_csv(GWAS_FILE, sep="\t", nrows=2).columns

assert (
    "panel_variant_id" in gwas_file_columns
), "The GWAS file must be the final imputed one using the TWAS imputation tools with column 'panel_variant_id'"

assert (
    "pvalue" in gwas_file_columns
), "The GWAS file must be the final imputed one using the TWAS imputation tools with column 'pvalue'"

assert (
    "zscore" in gwas_file_columns
), "The GWAS file must be the final imputed one using the TWAS imputation tools with column 'zscore'"

In [18]:
gwas_data = pd.read_csv(
    GWAS_FILE,
    sep="\t",
    usecols=["panel_variant_id", "pvalue", "zscore"],
)

In [19]:
gwas_data.shape

(8580225, 3)

In [20]:
gwas_data.head()

,panel_variant_id,zscore,pvalue
0,chr1_54490_G_A_b38,0.609075,0.542475
1,chr1_87021_T_C_b38,-0.940829,0.346793
2,chr1_263722_C_G_b38,0.562776,0.573587
3,chr1_594402_C_T_b38,0.183769,0.854195
4,chr1_630555_C_T_b38,-0.205482,0.837195


In [21]:
gwas_data.dropna().shape

(8578486, 3)

In [22]:
# remove SNPs with no results
gwas_data = gwas_data.dropna()

In [23]:
gwas_data.shape

(8578486, 3)

## Save GWAS variants

In [24]:
gwas_data.head()

,panel_variant_id,zscore,pvalue
0,chr1_54490_G_A_b38,0.609075,0.542475
1,chr1_87021_T_C_b38,-0.940829,0.346793
2,chr1_263722_C_G_b38,0.562776,0.573587
3,chr1_594402_C_T_b38,0.183769,0.854195
4,chr1_630555_C_T_b38,-0.205482,0.837195


In [25]:
# in eMERGE's results, some values here are repeated (will be removed later by taking the unique set of variant IDs).
gwas_data["panel_variant_id"].is_unique

True

In [26]:
gwas_variants_ids_set = frozenset(gwas_data["panel_variant_id"])
list(gwas_variants_ids_set)[:5]

['chr2_4052503_G_A_b38',
 'chr14_49009012_G_A_b38',
 'chr6_45585720_AG_A_b38',
 'chr11_102834927_C_G_b38',
 'chr2_2790519_G_A_b38']

In [27]:
with open(OUTPUT_DIR_BASE / "gwas_variant_ids.pkl", "wb") as handle:
    pickle.dump(gwas_variants_ids_set, handle, protocol=pickle.HIGHEST_PROTOCOL)

# TWAS

## Available tissues for eQTL model

In [28]:
prediction_model_tissues = conf.PHENOMEXCAN["PREDICTION_MODELS"][
    f"{EQTL_MODEL}_TISSUES"
].split(" ")

In [29]:
len(prediction_model_tissues)

49

In [30]:
prediction_model_tissues[:5]

['Thyroid',
 'Artery_Aorta',
 'Heart_Atrial_Appendage',
 'Liver',
 'Heart_Left_Ventricle']

## S-MultiXcan results

In [31]:
smultixcan_results = pd.read_csv(
    SMULTIXCAN_FILE, sep="\t", usecols=["gene", "gene_name", "pvalue", "n", "n_indep"]
)

In [32]:
smultixcan_results.shape

(22314, 5)

In [33]:
smultixcan_results = smultixcan_results.dropna()

In [34]:
smultixcan_results.shape

(22311, 5)

In [35]:
smultixcan_results = smultixcan_results.assign(
    gene_id=smultixcan_results["gene"].apply(lambda g: g.split(".")[0])
)

In [36]:
smultixcan_results.head()

,gene,gene_name,pvalue,n,n_indep,gene_id
0,ENSG00000196735.11,HLA-DQA1,7.348761e-41,44.0,6.0,ENSG00000196735
1,ENSG00000204290.10,BTNL2,1.946528e-35,21.0,10.0,ENSG00000204290
2,ENSG00000204287.13,HLA-DRA,1.368742e-34,31.0,6.0,ENSG00000204287
3,ENSG00000237541.3,HLA-DQA2,1.435612e-33,45.0,5.0,ENSG00000237541
4,ENSG00000179344.16,HLA-DQB1,5.862907e-33,49.0,3.0,ENSG00000179344


In [37]:
assert smultixcan_results["gene_id"].is_unique

### Get common genes with MultiPLIER

In [38]:
common_genes = set(multiplier_z_genes).intersection(
    set(smultixcan_results["gene_name"])
)

In [39]:
len(common_genes)

6445

In [40]:
sorted(list(common_genes))[:5]

['A2M', 'AAAS', 'AANAT', 'AARS', 'AARS2']

## Genes info

In [41]:
multiplier_gene_obj = {
    gene_name: Gene(name=gene_name)
    for gene_name in common_genes
    if gene_name in Gene.GENE_NAME_TO_ID_MAP
}

In [42]:
# delete common_genes, from now on, genes_info should be used for common genes
del common_genes

In [43]:
len(multiplier_gene_obj)

6445

In [44]:
assert multiplier_gene_obj["GAS6"].ensembl_id == "ENSG00000183087"

In [45]:
_gene_obj = list(multiplier_gene_obj.values())

genes_info = pd.DataFrame(
    {
        "name": [g.name for g in _gene_obj],
        "id": [g.ensembl_id for g in _gene_obj],
        "chr": [g.chromosome for g in _gene_obj],
        "band": [g.band for g in _gene_obj],
        "start_position": [g.get_attribute("start_position") for g in _gene_obj],
        "end_position": [g.get_attribute("end_position") for g in _gene_obj],
    }
)

In [46]:
genes_info = genes_info.assign(
    gene_length=genes_info.apply(
        lambda x: x["end_position"] - x["start_position"], axis=1
    )
)

In [47]:
genes_info.dtypes

name               object
id                 object
chr                object
band               object
start_position    float64
end_position      float64
gene_length       float64
dtype: object

In [48]:
_tmp = genes_info[genes_info.isna().any(axis=1)]
display(_tmp)
assert _tmp.shape[0] < 5

,name,id,chr,band,start_position,end_position,gene_length
3913,TMEM133,ENSG00000170647,None,None,NaN,NaN,NaN
4784,TBCE,ENSG00000116957,None,None,NaN,NaN,NaN


In [49]:
genes_info = genes_info.dropna()

In [50]:
genes_info["chr"] = genes_info["chr"].apply(pd.to_numeric, downcast="integer")
genes_info["start_position"] = genes_info["start_position"].astype(int)
genes_info["end_position"] = genes_info["end_position"].astype(int)
genes_info["gene_length"] = genes_info["gene_length"].astype(int)

In [51]:
genes_info.dtypes

name              object
id                object
chr                 int8
band              object
start_position     int64
end_position       int64
gene_length        int64
dtype: object

In [52]:
assert genes_info["name"].is_unique

In [53]:
assert genes_info["id"].is_unique

In [54]:
genes_info.shape

(6443, 7)

In [55]:
genes_info.head()

,name,id,chr,band,start_position,end_position,gene_length
0,MRPL44,ENSG00000135900,2,2q36.1,223957463,223967714,10251
1,H1FX,ENSG00000184897,3,3q21.3,129314771,129316286,1515
2,AGRN,ENSG00000188157,1,1p36.33,1020120,1056118,35998
3,DNAJB2,ENSG00000135924,2,2q35,219279267,219286900,7633
4,PISD,ENSG00000241878,22,22q12.2,31618491,31662432,43941


In [56]:
genes_info.sort_values("chr")

,name,id,chr,band,start_position,end_position,gene_length
2588,SORT1,ENSG00000134243,1,1p13.3,109309568,109397918,88350
4009,C4BPB,ENSG00000123843,1,1q32.1,207088860,207099993,11133
1583,COL9A2,ENSG00000049089,1,1p34.2,40300489,40317813,17324
4027,MTMR11,ENSG00000014914,1,1q21.2,149928651,149936879,8228
5487,CD1D,ENSG00000158473,1,1q23.1,158179947,158184896,4949
...,...,...,...,...,...,...,...
3389,SSTR3,ENSG00000278195,22,22q13.1,37204237,37212477,8240
2447,CSNK1E,ENSG00000213923,22,22q13.1,38290691,38318084,27393
5728,SEZ6L,ENSG00000100095,22,22q12.1,26169462,26383597,214135
5548,APOBEC3G,ENSG00000239713,22,22q13.1,39077067,39087743,10676


### Save

In [57]:
genes_info.to_pickle(OUTPUT_DIR_BASE / "genes_info.pkl")

## S-PrediXcan results

### Load results across all tissues

In [58]:
spredixcan_result_files = {
    t: SPREDIXCAN_FOLDER / SPREDIXCAN_FILE_PATTERN.format(tissue=t)
    for t in prediction_model_tissues
}

In [59]:
assert len(spredixcan_result_files) == len(prediction_model_tissues)
display(list(spredixcan_result_files.values())[:5])

[PosixPath('/opt/data/projects/asthma-copd/results/twas/spredixcan/GWAS_Asthma_only_GLM_SNPs_info0.7-gtex_v8-mashr-Thyroid.csv'),
 PosixPath('/opt/data/projects/asthma-copd/results/twas/spredixcan/GWAS_Asthma_only_GLM_SNPs_info0.7-gtex_v8-mashr-Artery_Aorta.csv'),
 PosixPath('/opt/data/projects/asthma-copd/results/twas/spredixcan/GWAS_Asthma_only_GLM_SNPs_info0.7-gtex_v8-mashr-Heart_Atrial_Appendage.csv'),
 PosixPath('/opt/data/projects/asthma-copd/results/twas/spredixcan/GWAS_Asthma_only_GLM_SNPs_info0.7-gtex_v8-mashr-Liver.csv'),
 PosixPath('/opt/data/projects/asthma-copd/results/twas/spredixcan/GWAS_Asthma_only_GLM_SNPs_info0.7-gtex_v8-mashr-Heart_Left_Ventricle.csv')]

In [60]:
# look at the structure of one result
pd.read_csv(spredixcan_result_files["Whole_Blood"]).head()

,gene,gene_name,zscore,effect_size,pvalue,var_g,pred_perf_r2,pred_perf_pval,pred_perf_qval,n_snps_used,n_snps_in_cov,n_snps_in_model,best_gwas_p,largest_weight
0,ENSG00000179344.16,HLA-DQB1,-12.301224,NaN,8.921179e-35,0.283759,NaN,NaN,NaN,3,3,3,4.004190e-34,0.772332
1,ENSG00000232629.8,HLA-DQB2,11.907670,NaN,1.079511e-32,1.363601,NaN,NaN,NaN,2,3,3,1.761500e-32,0.972033
2,ENSG00000237541.3,HLA-DQA2,10.481759,NaN,1.047767e-25,0.118803,NaN,NaN,NaN,2,4,4,4.049999e-21,0.486574
3,ENSG00000161405.16,IKZF3,-9.524108,NaN,1.664654e-21,0.002611,NaN,NaN,NaN,1,1,1,1.664654e-21,0.073940
4,ENSG00000073605.18,GSDMB,8.267584,NaN,1.367011e-16,0.098072,NaN,NaN,NaN,3,3,3,5.762121e-21,0.338834


In [61]:
assert all(f.exists() for f in spredixcan_result_files.values())

In [62]:
spredixcan_dfs = [
    pd.read_csv(
        f,
        usecols=[
            "gene",
            "zscore",
            "pvalue",
            "n_snps_used",
            "n_snps_in_model",
        ],
    )
    .dropna(subset=["gene", "zscore", "pvalue"])
    .assign(tissue=t)
    for t, f in spredixcan_result_files.items()
]

In [63]:
assert len(spredixcan_dfs) == len(prediction_model_tissues)

In [64]:
spredixcan_dfs = pd.concat(spredixcan_dfs)

In [65]:
assert spredixcan_dfs["tissue"].unique().shape[0] == len(prediction_model_tissues)

In [66]:
spredixcan_dfs.shape

(658530, 6)

In [67]:
spredixcan_dfs = spredixcan_dfs.assign(
    gene_id=spredixcan_dfs["gene"].apply(lambda g: g.split(".")[0])
)

In [68]:
spredixcan_dfs.head()

,gene,zscore,pvalue,n_snps_used,n_snps_in_model,tissue,gene_id
0,ENSG00000232629.8,12.131690,7.175314e-34,1,2,Thyroid,ENSG00000232629
1,ENSG00000196735.11,-11.318702,1.060298e-29,3,3,Thyroid,ENSG00000196735
2,ENSG00000179344.16,-10.453348,1.414416e-25,4,4,Thyroid,ENSG00000179344
3,ENSG00000237541.3,9.431310,4.049999e-21,1,3,Thyroid,ENSG00000237541
4,ENSG00000166949.15,-8.748763,2.157045e-18,3,3,Thyroid,ENSG00000166949


In [69]:
# leave only common genes
spredixcan_dfs = spredixcan_dfs[spredixcan_dfs["gene_id"].isin(set(genes_info["id"]))]

In [70]:
spredixcan_dfs.shape

(235569, 7)

### Count number of tissues available per gene

In [71]:
spredixcan_genes_n_models = spredixcan_dfs.groupby("gene_id")["tissue"].nunique()

In [72]:
spredixcan_genes_n_models

gene_id
ENSG00000000419     2
ENSG00000000938    36
ENSG00000000971    34
ENSG00000001084    32
ENSG00000001167    40
                   ..
ENSG00000278540    36
ENSG00000278828     4
ENSG00000278845    49
ENSG00000281005    49
ENSG00000282608    36
Name: tissue, Length: 6443, dtype: int64

In [73]:
# testing that in S-MultiXcan I get the same number of tissues per gene
_tmp_smultixcan_results_n_models = (
    smultixcan_results.set_index("gene_id")["n"].astype(int).rename("tissue")
)

_cg = _tmp_smultixcan_results_n_models.index.intersection(
    spredixcan_genes_n_models.index
)
_tmp_smultixcan_results_n_models = _tmp_smultixcan_results_n_models.loc[_cg]
_spredixcan = spredixcan_genes_n_models.loc[_cg]

assert _spredixcan.shape[0] == _tmp_smultixcan_results_n_models.shape[0]
assert _spredixcan.equals(_tmp_smultixcan_results_n_models.loc[_spredixcan.index])

### Get tissues available per gene

In [74]:
spredixcan_genes_models = spredixcan_dfs.groupby("gene_id")["tissue"].apply(
    lambda x: frozenset(x.tolist())
)

In [75]:
spredixcan_genes_models

gene_id
ENSG00000000419         (Brain_Substantia_nigra, Brain_Hypothalamus)
ENSG00000000938    (Adipose_Subcutaneous, Artery_Tibial, Brain_Hi...
ENSG00000000971    (Brain_Putamen_basal_ganglia, Artery_Tibial, B...
ENSG00000001084    (Brain_Putamen_basal_ganglia, Adipose_Subcutan...
ENSG00000001167    (Brain_Putamen_basal_ganglia, Adipose_Subcutan...
                                         ...                        
ENSG00000278540    (Brain_Putamen_basal_ganglia, Artery_Tibial, B...
ENSG00000278828    (Artery_Coronary, Cells_EBV-transformed_lympho...
ENSG00000278845    (Brain_Putamen_basal_ganglia, Adipose_Subcutan...
ENSG00000281005    (Brain_Putamen_basal_ganglia, Adipose_Subcutan...
ENSG00000282608    (Brain_Putamen_basal_ganglia, Adipose_Subcutan...
Name: tissue, Length: 6443, dtype: object

In [76]:
assert spredixcan_genes_n_models.shape[0] == spredixcan_genes_models.shape[0]

In [77]:
assert spredixcan_genes_n_models.index.equals(spredixcan_genes_models.index)

In [78]:
assert (spredixcan_genes_models.apply(len) <= len(prediction_model_tissues)).all()

In [79]:
spredixcan_genes_models.apply(len).describe()

count    6443.000000
mean       36.562005
std        12.886857
min         1.000000
25%        29.000000
50%        42.000000
75%        47.000000
max        49.000000
Name: tissue, dtype: float64

In [80]:
# testing that I obtained the right number of tissues
assert (
    spredixcan_genes_models.loc[spredixcan_genes_n_models.index]
    .apply(len)
    .equals(spredixcan_genes_n_models)
)

### Add gene name and set index

In [81]:
spredixcan_genes_models = spredixcan_genes_models.to_frame().reset_index()

In [82]:
spredixcan_genes_models.head()

,gene_id,tissue
0,ENSG00000000419,"(Brain_Substantia_nigra, Brain_Hypothalamus)"
1,ENSG00000000938,"(Adipose_Subcutaneous, Artery_Tibial, Brain_Hi..."
2,ENSG00000000971,"(Brain_Putamen_basal_ganglia, Artery_Tibial, B..."
3,ENSG00000001084,"(Brain_Putamen_basal_ganglia, Adipose_Subcutan..."
4,ENSG00000001167,"(Brain_Putamen_basal_ganglia, Adipose_Subcutan..."


In [83]:
spredixcan_genes_models = spredixcan_genes_models.assign(
    gene_name=spredixcan_genes_models["gene_id"].apply(
        lambda g: Gene.GENE_ID_TO_NAME_MAP[g]
    )
)

In [84]:
spredixcan_genes_models = spredixcan_genes_models[["gene_id", "gene_name", "tissue"]]

In [85]:
spredixcan_genes_models = spredixcan_genes_models.set_index("gene_id")

In [86]:
spredixcan_genes_models.head()

,gene_name,tissue
gene_id,,
ENSG00000000419,DPM1,"(Brain_Substantia_nigra, Brain_Hypothalamus)"
ENSG00000000938,FGR,"(Adipose_Subcutaneous, Artery_Tibial, Brain_Hi..."
ENSG00000000971,CFH,"(Brain_Putamen_basal_ganglia, Artery_Tibial, B..."
ENSG00000001084,GCLC,"(Brain_Putamen_basal_ganglia, Adipose_Subcutan..."
ENSG00000001167,NFYA,"(Brain_Putamen_basal_ganglia, Adipose_Subcutan..."


### Add number of tissues

In [87]:
spredixcan_genes_models = spredixcan_genes_models.assign(
    n_tissues=spredixcan_genes_models["tissue"].apply(len)
)

In [88]:
spredixcan_genes_models.head()

,gene_name,tissue,n_tissues
gene_id,,,
ENSG00000000419,DPM1,"(Brain_Substantia_nigra, Brain_Hypothalamus)",2
ENSG00000000938,FGR,"(Adipose_Subcutaneous, Artery_Tibial, Brain_Hi...",36
ENSG00000000971,CFH,"(Brain_Putamen_basal_ganglia, Artery_Tibial, B...",34
ENSG00000001084,GCLC,"(Brain_Putamen_basal_ganglia, Adipose_Subcutan...",32
ENSG00000001167,NFYA,"(Brain_Putamen_basal_ganglia, Adipose_Subcutan...",40


### Save

Here I quickly save these results to a file, given that the next steps (covariates) are slow to compute.

In [89]:
# this is important, other scripts depend on gene_name to be unique
assert spredixcan_genes_models["gene_name"].is_unique

In [90]:
assert not spredixcan_genes_models.isna().any(axis=None)

In [91]:
spredixcan_genes_models.to_pickle(OUTPUT_DIR_BASE / "gene_tissues.pkl")

## Add covariates based on S-PrediXcan results

This extend the previous file with more columns

### Get gene's objects

In [92]:
spredixcan_gene_obj = {
    gene_id: Gene(ensembl_id=gene_id) for gene_id in spredixcan_genes_models.index
}

In [93]:
len(spredixcan_gene_obj)

6443

### Count number of SNPs predictors used across tissue models

In [94]:
spredixcan_genes_sum_of_n_snps_used = (
    spredixcan_dfs.groupby("gene_id")["n_snps_used"].sum().rename("n_snps_used_sum")
)

In [95]:
spredixcan_genes_sum_of_n_snps_used

gene_id
ENSG00000000419     2
ENSG00000000938    40
ENSG00000000971    44
ENSG00000001084    46
ENSG00000001167    47
                   ..
ENSG00000278540    44
ENSG00000278828     5
ENSG00000278845    89
ENSG00000281005    81
ENSG00000282608    40
Name: n_snps_used_sum, Length: 6443, dtype: int64

In [96]:
# add sum of snps used to spredixcan_genes_models
spredixcan_genes_models = spredixcan_genes_models.join(
    spredixcan_genes_sum_of_n_snps_used
)

In [97]:
spredixcan_genes_models.shape

(6443, 4)

In [98]:
spredixcan_genes_models.head()

,gene_name,tissue,n_tissues,n_snps_used_sum
gene_id,,,,
ENSG00000000419,DPM1,"(Brain_Substantia_nigra, Brain_Hypothalamus)",2,2
ENSG00000000938,FGR,"(Adipose_Subcutaneous, Artery_Tibial, Brain_Hi...",36,40
ENSG00000000971,CFH,"(Brain_Putamen_basal_ganglia, Artery_Tibial, B...",34,44
ENSG00000001084,GCLC,"(Brain_Putamen_basal_ganglia, Adipose_Subcutan...",32,46
ENSG00000001167,NFYA,"(Brain_Putamen_basal_ganglia, Adipose_Subcutan...",40,47


### Count number of SNPs predictors in models across tissue models

In [99]:
spredixcan_genes_sum_of_n_snps_in_model = (
    spredixcan_dfs.groupby("gene_id")["n_snps_in_model"]
    .sum()
    .rename("n_snps_in_model_sum")
)

In [100]:
spredixcan_genes_sum_of_n_snps_in_model

gene_id
ENSG00000000419     2
ENSG00000000938    40
ENSG00000000971    44
ENSG00000001084    46
ENSG00000001167    48
                   ..
ENSG00000278540    44
ENSG00000278828     5
ENSG00000278845    91
ENSG00000281005    81
ENSG00000282608    40
Name: n_snps_in_model_sum, Length: 6443, dtype: int64

In [101]:
# add sum of snps in model to spredixcan_genes_models
spredixcan_genes_models = spredixcan_genes_models.join(
    spredixcan_genes_sum_of_n_snps_in_model
)

In [102]:
spredixcan_genes_models.shape

(6443, 5)

In [103]:
spredixcan_genes_models.head()

,gene_name,tissue,n_tissues,n_snps_used_sum,n_snps_in_model_sum
gene_id,,,,,
ENSG00000000419,DPM1,"(Brain_Substantia_nigra, Brain_Hypothalamus)",2,2,2
ENSG00000000938,FGR,"(Adipose_Subcutaneous, Artery_Tibial, Brain_Hi...",36,40,40
ENSG00000000971,CFH,"(Brain_Putamen_basal_ganglia, Artery_Tibial, B...",34,44,44
ENSG00000001084,GCLC,"(Brain_Putamen_basal_ganglia, Adipose_Subcutan...",32,46,46
ENSG00000001167,NFYA,"(Brain_Putamen_basal_ganglia, Adipose_Subcutan...",40,47,48


### Save

In [104]:
# this is important, other scripts depend on gene_name to be unique
assert spredixcan_genes_models["gene_name"].is_unique

In [105]:
assert not spredixcan_genes_models.isna().any(axis=None)

In [106]:
spredixcan_genes_models.to_pickle(OUTPUT_DIR_BASE / "gene_tissues.pkl")